2.16.1


In [17]:
import tensorflow as tf
import keras
from keras import layers, Model, Input
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from keras.optimizers import Adam
import numpy as np

In [18]:
# Create a base network
def create_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(64, (10, 10), activation='relu')(input)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    return Model(input, x)

input_shape = (105, 105, 1)
base_network = create_base_network(input_shape)

In [19]:
# Create and process a siamese network
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)


# Compute the L1 distance between the processed inputs
distance = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([processed_a, processed_b])
output = Dense(1, activation='sigmoid')(distance)

In [21]:
# compile the model
model = Model([input_a, input_b], output)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])